In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import nltk

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, accuracy_score, f1_score
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import confusion_matrix, roc_auc_score, recall_score, precision_score


In [2]:
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [3]:
from nltk.stem import PorterStemmer
porterStemmer = PorterStemmer()
import re

In [4]:
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer

In [5]:
stop_words=set(stopwords.words("english"))

In [6]:
data = pd.read_csv('/home/ridha/Downloads/Subtask-A-master/V1.4_Training.csv',header=None)

In [7]:
test_data=pd.read_csv('/home/ridha/Downloads/Subtask-A-master/SubtaskA_Trial_Test_Labeled.csv',encoding='ISO-8859-1')

In [8]:
data[:10]

,0,1,2
0,663_3,"""Please enable removing language code from the...",1
1,663_4,"""Note: in your .csproj file, there is a Suppor...",0
2,664_1,"""Wich means the new version not fully replaced...",0
3,664_2,"""Some of my users will still receive the old x...",0
4,664_3,"""The store randomly gives the old xap or the n...",0
5,664_4,"""My app has a WP7 version and a WP8 version XA...",0
6,664_5,"""The wp7 xap works only on WP7 and the wp8 xap...",0
7,665_1,"""Sometimes the Store gives the wrong wp7 xap v...",0
8,665_2,"""It should be an option to remove the ""ru"" lan...",1
9,665_3,"""Currently if you ever mistakenly selected a ""...",0


## tried to work with train set divided into train and val set

In [9]:

train, test = train_test_split(data, test_size=0.2, random_state=1)
X_train = train[1].values
X_test = test[1].values
y_train = train[2]
y_test = test[2]

## Building Pipeline for svm along with grid search cv

In [10]:
def tokenize(text): 
    return nltk.word_tokenize(text)

def stem(doc):
    return (stemmer.stem(w) for w in analyzer(doc))

en_stopwords = set(stopwords.words("english")) 

vectorizer = CountVectorizer(
    analyzer = 'word',
    tokenizer = tokenize,
    lowercase = True,
    ngram_range=(1, 1),
    stop_words = en_stopwords)

In [11]:
kfolds = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

In [12]:
np.random.seed(1)

pipeline_svm = make_pipeline(vectorizer, 
                            SVC(probability=True, kernel="rbf", class_weight="balanced"))

grid_svm = GridSearchCV(pipeline_svm,
                    param_grid = {'svc__C': [0.01, 0.1, 1]}, 
                    cv = kfolds,
                    scoring="roc_auc",
                    verbose=1,   
                    n_jobs=-1) 

grid_svm.fit(X_train, y_train)
grid_svm.score(X_test, y_test)

## getting the best params and best estimator out of grid search

In [74]:
 grid_svm.best_params_

In [13]:
grid_svm.best_score_

0.8084708142346081

In [15]:
a=grid_svm.best_estimator_

In [46]:
len(test_data[test_data['label']==1])

296

In [32]:
result=a.predict(test_data['sentence'])

In [30]:
def classwise(y_true, y_pred):
    return classification_report(y_true, y_pred)

## getting the classwise accuracy results

In [33]:
classwise(test_data['label'], result)

/home/ridha/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


'              precision    recall  f1-score   support\n\n           0       0.00      0.00      0.00       296\n           1       0.50      1.00      0.67       296\n\n   micro avg       0.50      0.50      0.50       592\n   macro avg       0.25      0.50      0.33       592\nweighted avg       0.25      0.50      0.33       592\n'

In [34]:
 accuracy = accuracy_score(result, test_data['label'])

In [35]:
accuracy

0.5

In [52]:
print(len(data[data[2]==1]))
print(len(data[2]))

2085
8500


## Running svm pipeline for entire data

In [14]:
pipeline_svm = make_pipeline(vectorizer, 
                            SVC(probability=True, kernel="rbf"))

In [15]:
pipeline_svm.fit(data[1], data[2])

/home/ridha/anaconda3/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'ll", "'re", "'s", "'ve", 'could', 'might', 'must', "n't", 'need', 'sha', 'wo', 'would'] not in stop_words.
  'stop_words.' % sorted(inconsistent))
/home/ridha/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


Pipeline(memory=None,
     steps=[('countvectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None,
        stop_words=...bf', max_iter=-1, probability=True, random_state=None,
  shrinking=True, tol=0.001, verbose=False))])

## Measuring accuracy

In [16]:
result1=pipeline_svm.predict(test_data['sentence'])

In [18]:
accuracy = accuracy_score(result1, test_data['label'])

In [19]:
print(accuracy)

0.5


## Working with glove dictionary

In [16]:
!pip install glove_python

    100% |████████████████████████████████| 266kB 739kB/s ta 0:00:01
  Running setup.py bdist_wheel for glove-python ... done
  Stored in directory: /home/ridha/.cache/pip/wheels/88/4b/6d/10c0d2ad32c9d9d68beec9694a6f0b6e83ab1662a90a089a4b
Successfully built glove-python


## defining my own preprocessing function

In [41]:
def Preprocessing(data):
    sent=nltk.sent_tokenize(data)
    token=[]
    for sentence in sent:
        sent1=sentence.casefold()
        words=sent1.split()
        for word in words: 
            word1=word.strip('<>()\.\,\':\"\?\![\]{}@\^#*;_\-\+=$')
            word2=porterStemmer.stem(word1)
            token.append(word2)
    return token

In [31]:
# words = []
# idx = 0
# word2idx = {}
# vectors = []
# with open('/home/ridha/Desktop/glove.6B.50d.txt', 'rb') as f:
#     for l in f:
#         line = l.decode().split()
#         word = line[0]
#         words.append(word)
#         word2idx[word] = idx
#         idx += 1
#         vect = np.array(line[1:]).astype(np.float)
#         vectors.append(vect)
    
# vectors = bcolz.carray(vectors[1:].reshape((400000, 50)), rootdir=f'{glove_path}/6B.50.dat', mode='w')
# vectors.flush()
# pickle.dump(words, open(f'{glove_path}/6B.50_words.pkl', 'wb'))
# pickle.dump(word2idx, open(f'{glove_path}/6B.50_idx.pkl', 'wb'))

In [32]:
# vectors = np.array(vectors)


In [33]:
# vectors=vectors.reshape(400000, 50)

In [34]:
# print(vectors.shape)
# glove = {w: vectors[word2idx[w]] for w in words}

In [35]:
# f= open('glovedict','wb')
# pickle.dump(glove,f)

In [36]:
# f.close()
# glove={}

In [37]:
f= open('glovedict','rb')
glove=pickle.load(f)

In [38]:
glove['file']

array([ 0.20346 , -0.36144 ,  1.065   ,  0.77532 ,  0.13116 ,  0.084812,
       -0.09789 ,  0.058289,  0.068911, -0.29628 , -0.47937 ,  0.24026 ,
        0.19721 ,  0.41073 ,  0.80101 , -0.21375 , -1.3117  , -0.88078 ,
        1.3917  , -0.86331 ,  0.13966 , -0.68027 ,  0.058374,  0.8234  ,
       -0.72267 , -1.1106  , -0.10921 , -0.27527 ,  0.035973, -0.70028 ,
        2.5397  , -1.268   , -0.95494 , -0.66966 , -0.43851 ,  0.27497 ,
        0.602   , -1.2052  ,  0.33294 , -0.96118 ,  1.2398  , -0.17181 ,
        0.043022,  1.1535  , -0.64861 ,  0.10567 ,  0.38906 ,  0.47547 ,
        0.015746,  0.017795])

##  Using keywords from baseline implementation adding extra importance for these words

In [39]:
keywords = ["suggest","recommend","hopefully","go for","request","it would be nice","adding","should come with","should be able","could come with", "i need" , "we need","needs", "would like to","would love to","allow","add"]



## Creating the sentenceVector for training data getting the vector representation of word from glove and simply adding for all words in sentence

In [62]:
a=list(data[1])
corpa=[]
for sent in a:
    corpa.append([word for word in Preprocessing(sent)])
corpa[1] 
sentenceVect=[]
for sent_token in corpa:
    vect=np.zeros(50)
    for token in sent_token:
        if glove.get(token) is not None :
            vect+= glove[token]
            if token in keywords:
                vect+= 0.3*glove[token]
    sentenceVect.append(vect)

In [63]:
clf = SVC(gamma='auto',kernel="rbf",C=1.0,class_weight={1: 3})

In [64]:
#sentenceVect[1]
clf.fit(sentenceVect,data[2])

SVC(C=1.0, cache_size=200, class_weight={1: 3}, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

## Training accuracy

In [65]:
result=clf.predict(sentenceVect)
accuracy = accuracy_score(result, data[2])
print(accuracy)

0.9910588235294118


## Creating test sentence vector  

In [66]:
a=list(test_data['sentence'])
corpa=[]
for sent in a:
    corpa.append([word for word in Preprocessing(sent)])
corpa[1] 
sentenceVect=[]
for sent_token in corpa:
    vect=np.zeros(50)
    for token in sent_token:
        if glove.get(token) is not None:
            vect+= glove[token]
            if token in keywords:
                vect+= 0.3*glove[token]
    sentenceVect.append(vect)

## getting results for the evaluation data

In [67]:
result1=clf.predict(sentenceVect)
accuracy = accuracy_score(result1, test_data['label'])

In [68]:
print(accuracy)

0.5287162162162162


In [73]:
#data[1]


In [69]:
# from glove import Corpus, Glove

# # creating a corpus object
# corpus = Corpus() 

# #training the corpus to generate the co occurence matrix which is used in GloVe
# corpus.fit(corpa, window=10)

# #creating a Glove object which will use the matrix created in the above lines to create embeddings
# #We can set the learning rate as it uses Gradient Descent and number of components

# glove = Glove(no_components=5, learning_rate=0.05)
 
# glove.fit(corpus.matrix, epochs=30, no_threads=4, verbose=True)
# glove.add_dictionary(corpus.dictionary)
# glove.save('glove.model')

In [70]:
# glove.add_dictionary(corpus.dictionary)

In [72]:
# glove.word_vectors[glove.dictionary['file']]

In [71]:
# generating sentence vectors
# len(glove.dictionary.keys())